In [1]:
using System;
using System.IO;
using System.Collections.Generic;
using System.Collections.Concurrent;
using System.Collections;
using System.Threading;
using System.Text.Json;

In [2]:
public class Debug : IDisposable
{
    public static bool IsEnabled {get; private set;} = false;

    public static void Print(string input)
    {
        if(IsEnabled)
        {
            Console.WriteLine(input);
        }
    }

    public Debug(bool enable = true)
    {
        IsEnabled = enable;
    }

    public void Dispose()
    {
        IsEnabled = false;
    }
}

In [3]:
var input = File.ReadAllLines("input.txt");

In [4]:
public record Unit
{
    public int Health {get; set;}
    public int Damage {get; set;}
    public int Armor {get; set;}
}

In [5]:
var boss = new Unit();

foreach(var line in input)
{
    var x = int.Parse(line.Split(':').Last());

    if(line.Contains("Hit"))
    {
        boss.Health = x;
    }
    else if(line.Contains("Damage"))
    {
        boss.Damage = x;
    }
    else if(line.Contains("Armor"))
    {
        boss.Armor = x;
    }
}

boss.Display();

Health,Damage,Armor
58,9,0


In [6]:
#nullable enable

public static class Pathfinder
{
    public static PathfinderResult Dijkstra(
        BaseNode start)
    {
        long visited = 0;

        var costs = new Dictionary<ulong, int>();
        costs[start.Id] = start.Cost;
       
        var q = new PriorityQueue<BaseNode, int>();
        q.Enqueue(start, 0);

        while (q.TryDequeue(out var u, out var c))
        {
            visited++;

            if(visited % 10000 == 0)
            {
                Debug.Print($"Visited {visited} nodes");
            }
            
            if (u.IsGoal)
            {
                return new PathfinderResult
                {
                    Success = true,
                    GoalNode = u,
                    Visited = visited,
                    Total = c
                };
            }

            foreach (var v in u.Neighbours())
            {
                var t = c + v.Cost;

                var h = v.Id;
                if (costs.TryGetValue(h, out var cachedCost))
                {
                    if (cachedCost <= t)
                    {
                        continue;
                    }
                }

                costs[h] = t;
                
                v.Previous = u;
                q.Enqueue(v, t);
            }
        }

        return new PathfinderResult
        {
            Success = false,
            Visited = visited
        };
    }

    public static List<BaseNode> ReconstructPath<T>(BaseNode node)
    {
        var path = new List<BaseNode>();
        BaseNode? current = node;
        while (current is not null)
        {
            path.Insert(0, current);
            current = current.Previous;
        }

        return path;
    }
}

public interface BaseNode
{
    int Cost { get; init; }
    BaseNode? Previous { get; set; }
    IEnumerable<BaseNode> Neighbours();
    bool IsGoal { get; }
    ulong Id { get; }
}

public record PathfinderResult
{
    public bool Success { get; init; }
    public BaseNode? GoalNode { get; init; }
    public long Visited { get; set; }
    public long Total {get; set;}
}

In [12]:
public record GameState
{
    public ulong Id => ArmorDuration | (PoisonDuration << 4) | (RechargeDuration << 8) | (PlayerHealth << 12) | (BossHealth << 19) | (Mana << 27);
    public ulong BossHealth { get; set; }
    public ulong PlayerHealth { get; set; }
    public ulong Mana { get; set; }
    public ulong BossDamage { get; set; }
    public ulong ArmorDuration { get; set; }
    public ulong PoisonDuration { get; set; }
    public ulong RechargeDuration { get; set; }

    public GameState Next => this with {
        PlayerHealth = PlayerHealth - (BossDamage - (ulong) (ArmorDuration > 1 ? 7 : 0)),
        BossHealth = BossHealth - (ulong) (PoisonDuration > 0 ? 3 : 0) - (ulong) (PoisonDuration > 1 ? 3 : 0),
        Mana = Mana + (ulong) (RechargeDuration > 1 ? 101 : 0)
    };
} 

In [11]:
#nullable enable

public record GameNode : BaseNode
{
    public ulong Id => PreState.Id;
    
    public int Cost { get; init; }
    public BaseNode? Previous { get; set; }
    public ulong BossDamage { get; init; }
    public GameState PreState {get; init;}
    public GameState PostState {
        get
        {
            if(_postState is null)
            {
                _postState = PreState with {
                    
                };
            }

            return _postState;
        }
    }

    private GameState? _postState;
    

    public bool IsGoal {
        get {
            return false;
        }
    }

    public IEnumerable<BaseNode> Neighbours() {
        var neighbours = new List<BaseNode>();

        return neighbours;
    }

    public string Items {get; init;} = "";
    
}

In [9]:
var id = new GameState{
    InitialBossHealth = 128,
    InitialPlayerHealth = 64,
    InitialMana = 500000,
    ArmorDuration = 8,
    RechargeDuration = 8,
    PoisonDuration = 8
}.Id;

Console.WriteLine(Convert.ToString((long) id, 2));
Console.WriteLine(Convert.ToString((long) id, 2).Length);

1111010000100100000100000001000000100010001000
46
